In [1]:
# %%
import pandas as pd
import networkx as nx
import logging
from random import shuffle
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import random
import string
from itertools import combinations
import pickle
import gc
import os
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer

#os.environ['WANDB_MODE'] = 'disabled'
# set current directory
os.chdir("/cs/student/projects1/aibh/2024/cbaumgar/MSC_THESIS")

# %%
import os
import shutil
import random
import string
import networkx as nx
from tqdm import tqdm
import numpy as np


/cs/student/projects1/aibh/2024/cbaumgar/.venv_fresh/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
/cs/student/projects1/aibh/2024/cbaumgar/.venv_fresh/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# --- 1. UTILITY FUNCTIONS ---

def generate_random_names(count):
    """Generates a list of unique random 2-letter names."""
    names = set()
    while len(names) < count:
        names.add(''.join(random.choices(string.ascii_lowercase, k=2)))
    return list(names)

def get_grid_graph(nodes, size=4):
    """
    Creates a grid graph of a given size with the provided node names.
    Default size is set to 4x4.
    """
    if len(nodes) != size * size:
        raise ValueError(f"Incorrect number of nodes. Expected {size*size}, got {len(nodes)}")
    random.shuffle(nodes)
    G = nx.DiGraph()
    for r in range(size):
        for c in range(size):
            idx = r * size + c
            u = nodes[idx]
            # Add edges with directions
            if c < size - 1: G.add_edge(u, nodes[idx + 1], direction='EAST')
            if c > 0: G.add_edge(u, nodes[idx - 1], direction='WEST')
            if r < size - 1: G.add_edge(u, nodes[idx + size], direction='SOUTH')
            if r > 0: G.add_edge(u, nodes[idx - size], direction='NORTH')
    return G

def generate_single_random_walk(G, min_walk_length, max_walk_length):
    """
    Generates one random walk on the graph with a length randomly chosen
    from the specified range [min_walk_length, max_walk_length].
    """
    nodes = list(G.nodes)
    random.shuffle(nodes)
    if not nodes:
        return []
    
    # *** KEY CHANGE: Choose a random length for this specific walk ***
    walk_length = random.randint(min_walk_length, max_walk_length)
        
    walk = []
    current_node = random.choice(nodes)
    walk.append(current_node)
    
    while len(walk) < walk_length:
        neighbors = list(G.successors(current_node))
        if not neighbors:
            break  # End walk if at a dead end
        next_node = random.choice(neighbors)
        walk.append(next_node)
        current_node = next_node
        
    return walk

def walk_to_string(walk, G):
    """Converts a list of nodes into a 'NODE DIR NODE...' string."""
    if not walk or len(walk) < 2:
        return "" # Return empty string for trivial walks
        
    path_segments = []
    for i in range(len(walk) - 1):
        node1 = walk[i]
        node2 = walk[i+1]
        try:
            direction = G.edges[node1, node2]['direction']
            path_segments.append(f"{node1} {direction}")
        except KeyError:
            print(f"Warning: Edge ({node1}, {node2}) missing from graph.")
            continue

    return " ".join(path_segments) + " " + walk[-1]

# --- 2. CORE DATA GENERATION LOGIC ---

def generate_and_save_passive_walks(num_walks, min_walk_length, max_walk_length, grid_size, output_file_path, description=""):
    """
    Generates and saves a dataset of random walks for passive learning.
    For each walk, a new graph with new random node names is created.
    """
    print(f"--- Generating {num_walks} walks for {description} ---")
    
    num_nodes = grid_size * grid_size
    
    with open(output_file_path, "w", encoding="utf-8") as f:
        for _ in tqdm(range(num_walks), desc=f"Generating {description}"):
            # 1. Create a new graph with fresh random names for each walk
            node_names = generate_random_names(num_nodes)
            G = get_grid_graph(node_names, size=grid_size)
            
            # 2. Generate a single random walk on this new graph with variable length
            walk_nodes = generate_single_random_walk(G, min_walk_length, max_walk_length)
            
            # 3. Convert the walk to the required string format
            walk_str = walk_to_string(walk_nodes, G)
            
            # 4. Write to file if it's a valid, non-empty walk
            if walk_str:
                f.write(walk_str + "\n")

    print(f"Successfully saved {num_walks} walks to {output_file_path}")


In [3]:

# # %%
# # --- 3. MAIN EXECUTION BLOCK ---

# if __name__ == "__main__":
#     # --- Configuration ---
#     NUM_TRAIN_WALKS = 1000000
#     NUM_TEST_WALKS = 10000
    
#     # *** KEY CHANGE: Define min/max walk lengths and set grid size to 4 ***
#     MIN_WALK_LENGTH = 120
#     MAX_WALK_LENGTH = 120
#     GRID_SIZE = 4 # Sticking to 4x4 grids as requested

#     # Set a seed for reproducibility
#     random.seed(42)
#     np.random.seed(42)

#     # --- Setup Output Directory ---
#     output_dir = "fv1_data" # More descriptive name
#     if os.path.exists(output_dir):
#         print(f"Removing existing directory: {output_dir}")
#         shutil.rmtree(output_dir)
#     print(f"Creating new directory: {output_dir}")
#     os.makedirs(output_dir)

#     # --- Generate Training Data ---
#     train_file_path = os.path.join(output_dir, "train.txt")
#     generate_and_save_passive_walks(
#         num_walks=NUM_TRAIN_WALKS,
#         min_walk_length=MIN_WALK_LENGTH,
#         max_walk_length=MAX_WALK_LENGTH,
#         grid_size=GRID_SIZE,
#         output_file_path=train_file_path,
#         description="Training Set"
#     )

#     # --- Generate Testing Data ---
#     test_file_path = os.path.join(output_dir, "test.txt")
#     generate_and_save_passive_walks(
#         num_walks=NUM_TEST_WALKS,
#         min_walk_length=MIN_WALK_LENGTH,
#         max_walk_length=MAX_WALK_LENGTH,
#         grid_size=GRID_SIZE,
#         output_file_path=test_file_path,
#         description="Testing Set"
#     )

#     print(f"\n--- Passive walk data generation finished successfully! ---")
#     print(f"Data saved in the '{output_dir}' directory.")



In [4]:
# import os

# # --- Configuration ---
# SOURCE_DIR = "fv1_data"
# TARGET_DIR = "fv1_data_with_eos"
# FILES_TO_PROCESS = ["train.txt", "test.txt"]
# EOS_TOKEN = "[EOS]"

# # --- Create the new directory ---
# os.makedirs(TARGET_DIR, exist_ok=True)
# print(f"Created directory: {TARGET_DIR}")

# for filename in FILES_TO_PROCESS:
#     source_path = os.path.join(SOURCE_DIR, filename)
#     target_path = os.path.join(TARGET_DIR, filename)
    
#     print(f"Processing {source_path} -> {target_path}...")
    
#     with open(source_path, 'r', encoding='utf-8') as f_in, \
#          open(target_path, 'w', encoding='utf-8') as f_out:
        
#         for line in f_in:
#             # Strip the original newline and append the EOS token and a new newline
#             stripped_line = line.strip()
#             if stripped_line: # Ensure we don't just write EOS for empty lines
#                 f_out.write(f"{stripped_line} {EOS_TOKEN}\n")

# print("\nProcessing complete.")
# print(f"New data files with EOS tokens are saved in '{TARGET_DIR}'.")

In [5]:
# %%
class GPT:

    def __init__(self, base_model=None, base_model_name='gpt2', vocab_size=100):
        self.base_model = base_model
        self.base_model_name = base_model_name
        self.vocab_size = vocab_size

        if self.base_model is not None:
            self.tokenizer = GPT2Tokenizer.from_pretrained(base_model)
            self.model = GPT2LMHeadModel.from_pretrained(base_model)
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def continue_input(self, input_sequence, max_new_tokens=5, num_return_sequences=1, no_repeat_ngram_size=0,
                       do_sample=False, temperature=0.7, num_beams=1):
        input_ids = self.tokenizer.encode(input_sequence, return_tensors='pt')

        # Generate text
        output = self.model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            num_return_sequences=num_return_sequences,
            num_beams=num_beams,
            no_repeat_ngram_size=no_repeat_ngram_size,
            do_sample=do_sample,
            temperature=temperature,
        )

        # Decode the output
        sequence = output[0].tolist()
        text = self.tokenizer.decode(sequence)
        return text

# %%
def load_pkl(pth):
    with open(pth, 'rb') as f:
        d = pickle.load(f)
    return d

def is_valid_path(sequence, graphs):
    # Split the sequence into parts
    parts = sequence.split()

    # Extract nodes and edges; nodes are at even indices, edges at odd indices
    nodes = parts[::2]
    edges = parts[1::2]

    # Convert edges to a lowercase version for comparison (assuming all edges in graphs are lowercase)
    edges = [edge.lower() for edge in edges]

    # Iterate over each graph to check if the path exists
    for graph in graphs:
        path_exists = True
        for i in range(len(nodes) - 1):
            # Check if the current graph has the edge between the current node and the next node
            if not graph.has_edge(nodes[i], nodes[i+1]):
                path_exists = False
                break

        # If path exists in the current graph, return True
        if path_exists:
            return True

    # If none of the graphs contain the path, return False
    return False


In [6]:
# from transformers import GPT2Tokenizer
# import os

# TOKENIZER_PATH = "foraging_tokenizer" # Assuming this path contains your tokenizer files

# print(f"Loading custom tokenizer from: {TOKENIZER_PATH}")
# my_custom_tokenizer = GPT2Tokenizer.from_pretrained(TOKENIZER_PATH)

# print("\n--- Verification ---")

# # Case 1: Word with a leading space (as if it were a normal word)
# text_with_space = "Go NORTH"
# encoded_with_space = my_custom_tokenizer.encode(text_with_space, add_special_tokens=False)
# decoded_with_space = my_custom_tokenizer.decode(encoded_with_space)
# tokens_with_space = [my_custom_tokenizer.decode([t]) for t in encoded_with_space]

# print(f"\nOriginal text: '{text_with_space}'")
# print(f"Encoded IDs: {encoded_with_space}")
# print(f"Decoded back: '{decoded_with_space}'")
# print(f"Individual tokens: {tokens_with_space}")


# # Case 2: Word without a leading space (start of sentence or after punctuation)
# text_no_space = "NORTH pole"
# encoded_no_space = my_custom_tokenizer.encode(text_no_space, add_special_tokens=False)
# decoded_no_space = my_custom_tokenizer.decode(encoded_no_space)
# tokens_no_space = [my_custom_tokenizer.decode([t]) for t in encoded_no_space]

# print(f"\nOriginal text: '{text_no_space}'")
# print(f"Encoded IDs: {encoded_no_space}")
# print(f"Decoded back: '{decoded_no_space}'")
# print(f"Individual tokens: {tokens_no_space}")

# # Case 3: A word that is part of a larger word
# text_part_of_word = "NORTHERN"
# encoded_part_of_word = my_custom_tokenizer.encode(text_part_of_word, add_special_tokens=False)
# decoded_part_of_word = my_custom_tokenizer.decode(encoded_part_of_word)
# tokens_part_of_word = [my_custom_tokenizer.decode([t]) for t in encoded_part_of_word]

# print(f"\nOriginal text: '{text_part_of_word}'")
# print(f"Encoded IDs: {encoded_part_of_word}")
# print(f"Decoded back: '{decoded_part_of_word}'")
# print(f"Individual tokens: {tokens_part_of_word}")


# # Inspecting the vocabulary mapping for "NORTH"
# print(f"\nID for 'NORTH': {my_custom_tokenizer.get_vocab().get('NORTH')}")
# print(f"ID for 'ĠNORTH': {my_custom_tokenizer.get_vocab().get('ĠNORTH')}") # Should be None or a different ID if 'ĠNORTH' was in vocab

In [7]:

# # %%
# from transformers import AutoConfig, GPT2Tokenizer
# import os

# # --- Configuration ---
# TOKENIZER_PATH = "foraging_tokenizer"
# CONFIG_SAVE_PATH = "foraging_chunked_config"

# # 1. Load the custom tokenizer to get its properties
# print(f"Loading custom tokenizer from: {TOKENIZER_PATH}")
# my_custom_tokenizer = GPT2Tokenizer.from_pretrained(TOKENIZER_PATH)

# # 2. Load the base gpt2 config as a blueprint
# print("Loading base gpt2 config...")
# config = AutoConfig.from_pretrained('gpt2')

# # 3. CRITICAL: Override the config with our custom tokenizer's properties
# # --- IMPROVEMENT: Use len() for the most reliable vocab size ---
# print(f"Overriding config vocab_size. Old: {config.vocab_size}, New: {len(my_custom_tokenizer)}")
# config.vocab_size = len(my_custom_tokenizer)

# # Also update the special token IDs
# print("Updating special token IDs in the config...")
# config.bos_token_id = my_custom_tokenizer.bos_token_id
# config.eos_token_id = my_custom_tokenizer.eos_token_id

# # --- IMPROVEMENT: Explicitly set the pad_token_id ---
# # This ensures full compatibility with the Trainer and DataCollator.
# config.pad_token_id = my_custom_tokenizer.pad_token_id
# print(f"Set pad_token_id to: {config.pad_token_id}")
# # ----------------------------------------------------

# # 4. Save the new, modified config to its own directory
# os.makedirs(CONFIG_SAVE_PATH, exist_ok=True)
# config.save_pretrained(CONFIG_SAVE_PATH)

# print(f"\nCustom model config saved successfully to: {CONFIG_SAVE_PATH}")


In [12]:
from transformers import GPT2Tokenizer

# --- Load your custom tokenizer ---
TOKENIZER_PATH = "foraging_tokenizer"
tokenizer = GPT2Tokenizer.from_pretrained(TOKENIZER_PATH)

# --- Test input sequence ---
test_text = "eb EAST lt NORTH yz NORTH mb NORTH jf SOUTH mb WEST rr NORTH he SOUTH rr SOUTH no NORTH rr [START_NODE] jf [GOAL] eb [PLAN] jf SOUTH mb SOUTH yz SOUTH lt WEST eb [EOS]"

# --- Encode the sequence ---
encoded = tokenizer.encode(test_text, add_special_tokens=True)
print("Encoded IDs:", encoded)

# --- Decode back to text ---
decoded = tokenizer.decode(encoded)
print("Decoded text:", decoded)

# --- Extra: view tokens step by step ---
tokens = tokenizer.convert_ids_to_tokens(encoded)
print("Tokenized sequence:", tokens)


Encoded IDs: [1638, 282, 420, 276, 484, 276, 488, 276, 768, 272, 488, 279, 849, 276, 637, 272, 849, 272, 470, 276, 849, 231, 8, 768, 231, 9, 497, 231, 5, 768, 272, 488, 272, 484, 272, 420, 279, 497, 231, 2]
Decoded text: eb EAST lt NORTH yz NORTH mb NORTH jf SOUTH mb WEST rr NORTH he SOUTH rr SOUTH no NORTH rr [START_NODE] jf [GOAL] eb [PLAN] jf SOUTH mb SOUTH yz SOUTH lt WEST eb [EOS]
Tokenized sequence: ['eb', 'ĠEAST', 'Ġlt', 'ĠNORTH', 'Ġyz', 'ĠNORTH', 'Ġmb', 'ĠNORTH', 'Ġjf', 'ĠSOUTH', 'Ġmb', 'ĠWEST', 'Ġrr', 'ĠNORTH', 'Ġhe', 'ĠSOUTH', 'Ġrr', 'ĠSOUTH', 'Ġno', 'ĠNORTH', 'Ġrr', 'Ġ', '[START_NODE]', 'Ġjf', 'Ġ', '[GOAL]', 'Ġeb', 'Ġ', '[PLAN]', 'Ġjf', 'ĠSOUTH', 'Ġmb', 'ĠSOUTH', 'Ġyz', 'ĠSOUTH', 'Ġlt', 'ĠWEST', 'Ġeb', 'Ġ', '[EOS]']


In [8]:

# %%
import os

# Define the path for new cache directory 
project_cache_path = os.path.join(os.getcwd(), '.cache', 'huggingface_datasets')
os.makedirs(project_cache_path, exist_ok=True)

# Set the environment variable to tell the 'datasets' library to use this new path
os.environ['HF_DATASETS_CACHE'] = project_cache_path

print(f"Hugging Face datasets cache is now set to: {os.environ['HF_DATASETS_CACHE']}")

wandb_path = os.path.join(os.getcwd(), 'wandb_local_runs')
os.makedirs(wandb_path, exist_ok=True)
# Set the environment variable to tell 'wandb' to use this new path
os.environ['WANDB_DIR'] = wandb_path
print(f"Wandb local run directory set to: {os.environ['WANDB_DIR']}")

import os
os.environ["WANDB__SERVICE_WAIT"] = "120"


Hugging Face datasets cache is now set to: /cs/student/projects1/aibh/2024/cbaumgar/MSC_THESIS/.cache/huggingface_datasets
Wandb local run directory set to: /cs/student/projects1/aibh/2024/cbaumgar/MSC_THESIS/wandb_local_runs


In [9]:

# %%
def train_model_script(num_epochs=1,
                       output_dir='fv1_model',
                       lr=5e-05, wandb_run_id=None):
    gc.collect()
    try:
        wandb.login()
    except Exception as e:
        print(f"Could not log in to wandb: {e}")

    if wandb_run_id:
        print(f"Attempting to resume W&B run with ID: {wandb_run_id}")
        os.environ['WANDB_RESUME'] = "allow"
        os.environ['WANDB_RUN_ID'] = wandb_run_id

    os.environ['WANDB_PROJECT'] = 'foraging_models'

    # --- Hyperparameters ---
    save_steps = 500
    save_total_limit = 20
    weight_decay = 0.1
    beta2 = 0.98
    

    # --- Define Clean Paths ---
    data_dir = 'fv1_data'
    tokenizer_dir = 'foraging_tokenizer'
    config_dir = 'foraging_chunked_config'

    train_path = f'./{data_dir}/train.txt'
    test_path = f'./{data_dir}/test.txt'
    
    training_script_path = './run_clm.py'

    command = f"""
    python3 {training_script_path} \\
        --config_name {config_dir} \\
        --tokenizer_name {tokenizer_dir} \\
        --train_file {train_path} \\
        --validation_file {test_path} \\
        --do_train \\
        --do_eval \\
        --output_dir {output_dir} \\
        --per_device_train_batch_size 16 \\
        --per_device_eval_batch_size 16 \\
        --num_train_epochs {num_epochs} \\
        --save_strategy 'steps' \\
        --save_steps {save_steps} \\
        --save_total_limit {save_total_limit} \\
        --eval_strategy 'steps' \\
        --eval_steps 500 \\
        --warmup_steps 0 \\
        --learning_rate {lr} \\
        --report_to 'wandb' \\
        --lr_scheduler_type cosine \\
        --weight_decay {weight_decay} \\
        --adam_beta2 {beta2} \\
        --dataloader_num_workers 0 \\
        --overwrite_cache \
    """

    print("--- Running Command ---")
    print(command)
    print("-----------------------")

    os.system(command)
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"



In [10]:
#!rm -rf fv1_model/
!mkdir fv1_model_chunked/

mkdir: cannot create directory ‘fv1_model_chunked/’: File exists


In [11]:
train_model_script(num_epochs=8,
                   output_dir='fv1_model_chunked',
                   lr=1e-04, wandb_run_id="ci75uvds")  # Replace None with actual run ID to resume


Could not log in to wandb: name 'wandb' is not defined
Attempting to resume W&B run with ID: ci75uvds
--- Running Command ---

    python3 ./run_clm.py \
        --config_name foraging_chunked_config \
        --tokenizer_name foraging_tokenizer \
        --train_file ./fv1_data/train.txt \
        --validation_file ./fv1_data/test.txt \
        --do_train \
        --do_eval \
        --output_dir fv1_model_chunked \
        --per_device_train_batch_size 16 \
        --per_device_eval_batch_size 16 \
        --num_train_epochs 8 \
        --save_strategy 'steps' \
        --save_steps 500 \
        --save_total_limit 20 \
        --eval_strategy 'steps' \
        --eval_steps 500 \
        --warmup_steps 0 \
        --learning_rate 0.0001 \
        --report_to 'wandb' \
        --lr_scheduler_type cosine \
        --weight_decay 0.1 \
        --adam_beta2 0.98 \
        --dataloader_num_workers 0 \
        --overwrite_cache     
-----------------------
09/05/2025 03:38:11 - WARNING - 

Using custom data configuration default-d05b448f19c4b626
Found cached dataset text (/cs/student/projects1/aibh/2024/cbaumgar/MSC_THESIS/.cache/huggingface_datasets/text/default-d05b448f19c4b626/0.0.0/37eaf37ac90527a7fd768c94b312ee84f8815c9b7ac00acf81c1c364e8392f99)
[INFO|configuration_utils.py:763] 2025-09-05 03:38:12,280 >> loading configuration file foraging_chunked_config/config.json
[INFO|configuration_utils.py:839] 2025-09-05 03:38:12,281 >> Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 3,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 0,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summa

09/05/2025 03:38:12 - INFO - datasets.builder - Using custom data configuration default-d05b448f19c4b626
09/05/2025 03:38:12 - INFO - datasets.builder - Found cached dataset text (/cs/student/projects1/aibh/2024/cbaumgar/MSC_THESIS/.cache/huggingface_datasets/text/default-d05b448f19c4b626/0.0.0/37eaf37ac90527a7fd768c94b312ee84f8815c9b7ac00acf81c1c364e8392f99)
09/05/2025 03:38:12 - INFO - __main__ - Training new model from scratch - Total size=83.08M params
09/05/2025 03:38:12 - INFO - datasets.arrow_dataset - Caching processed dataset at /cs/student/projects1/aibh/2024/cbaumgar/MSC_THESIS/.cache/huggingface_datasets/text/default-d05b448f19c4b626/0.0.0/37eaf37ac90527a7fd768c94b312ee84f8815c9b7ac00acf81c1c364e8392f99/cache-0a0d076d97ff9d63.arrow


Running tokenizer on dataset: 100%|██████████| 1000000/1000000 [00:00<?, ? examples/s]Caching processed dataset at /cs/student/projects1/aibh/2024/cbaumgar/MSC_THESIS/.cache/huggingface_datasets/text/default-d05b448f19c4b626/0.0.0/37eaf37ac90527a7fd768c94b312ee84f8815c9b7ac00acf81c1c364e8392f99/cache-0a0d076d97ff9d63.arrow
Running tokenizer on dataset: 2000000 examples [01:21, 12298.53 examples/s]           
Running tokenizer on dataset: 100%|██████████| 10000/10000 [00:00<?, ? examples/s]Caching processed dataset at /cs/student/projects1/aibh/2024/cbaumgar/MSC_THESIS/.cache/huggingface_datasets/text/default-d05b448f19c4b626/0.0.0/37eaf37ac90527a7fd768c94b312ee84f8815c9b7ac00acf81c1c364e8392f99/cache-51fd396e9f0dc66b.arrow
Running tokenizer on dataset: 12000 examples [00:00, 12873.03 examples/s]         

09/05/2025 03:39:34 - INFO - datasets.arrow_dataset - Caching processed dataset at /cs/student/projects1/aibh/2024/cbaumgar/MSC_THESIS/.cache/huggingface_datasets/text/default-d05b448f19c4b626/0.0.0/37eaf37ac90527a7fd768c94b312ee84f8815c9b7ac00acf81c1c364e8392f99/cache-51fd396e9f0dc66b.arrow


...

{'train_runtime': 26439.9654, 'train_samples_per_second': 70.802, 'train_steps_per_second': 4.425, 'train_loss': 0.30346304034371663, 'epoch': 8.0}


[INFO|configuration_utils.py:491] 2025-09-05 11:03:15,692 >> Configuration saved in fv1_model_chunked/config.json
[INFO|configuration_utils.py:826] 2025-09-05 11:03:15,720 >> Configuration saved in fv1_model_chunked/generation_config.json
[INFO|modeling_utils.py:4297] 2025-09-05 11:03:21,511 >> Model weights saved in fv1_model_chunked/model.safetensors
[INFO|tokenization_utils_base.py:2563] 2025-09-05 11:03:21,542 >> tokenizer config file saved in fv1_model_chunked/tokenizer_config.json
[INFO|tokenization_utils_base.py:2572] 2025-09-05 11:03:21,564 >> Special tokens file saved in fv1_model_chunked/special_tokens_map.json


***** train metrics *****
  epoch                    =         8.0
  total_flos               = 911091796GF
  train_loss               =      0.3035
  train_runtime            =  7:20:39.96
  train_samples            =      234000
  train_samples_per_second =      70.802
  train_steps_per_second   =       4.425
09/05/2025 11:03:21 - INFO - __main__ - *** Evaluate ***


[INFO|trainer.py:4623] 2025-09-05 11:03:21,729 >> 
***** Running Evaluation *****
[INFO|trainer.py:4625] 2025-09-05 11:03:21,730 >>   Num examples = 2340
[INFO|trainer.py:4628] 2025-09-05 11:03:21,730 >>   Batch size = 16
100%|██████████| 147/147 [00:35<00:00,  4.17it/s]
[INFO|modelcard.py:456] 2025-09-05 11:03:57,357 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.5937033695098212}]}


***** eval metrics *****
  epoch                   =        8.0
  eval_accuracy           =     0.5937
  eval_loss               =     1.0604
  eval_runtime            = 0:00:35.52
  eval_samples            =       2340
  eval_samples_per_second =     65.868
  eval_steps_per_second   =      4.138
  perplexity              =     2.8876
wandb: 
wandb: 🚀 View run fv1_model_chunked at: https://wandb.ai/carobgt-ucl/foraging_models/runs/ci75uvds
wandb: Find logs at: wandb_local_runs/wandb/run-20250905_034235-ci75uvds/logs
